In [1]:
import praw
import pandas as pd
import json
from tqdm.notebook import tqdm

client_id = 'QXgnNilxAJoowxnJqYSUmA'
client_secret = 'oigCTLqOddP8NkRg_CHbFDI0SElr3w'

reddit = praw.Reddit(client_id = client_id,
                     client_secret = client_secret,
                     user_agent = 'IS 596')

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


In [2]:
subreddit_list = ["politics", "worldnews", "conservative",
                  "conspiracy", "NeutralPolitics", "AMA"]

In [3]:
def subreddit_crawler(sub, limit = 10):
    subreddit = reddit.subreddit(sub)
    post_list = []
    submission_fields = ['id', 'author', 'title', 'selftext', 'num_comments',
              'score', 'upvote_ratio', 'created_utc', 'url']
    for submission in subreddit.top(limit = limit, time_filter = "year"):
        to_dict = vars(submission)
        sub_dict = {field:to_dict[field] for field in submission_fields}
        post_list.append(sub_dict)
    
    subs_data = pd.DataFrame(data = post_list, columns=submission_fields)
    file_name = "data/" + "submission_" + sub + ".csv"
    subs_data.to_csv(file_name, index = False)

In [4]:
def data_processing(sub, field = "submission"):
    file_name = "data/" + field + "_" + sub + ".csv"
    df = pd.read_csv(file_name,index_col=0)
    print("subreddit : " + sub)
    print(df.describe())
    print(df.info())

In [5]:
def comment_parser(sub, checkpoint = 0):
    file_name = "data/" + "submission_" + sub + ".csv"
    df = pd.read_csv(file_name)
    
    sublist = df["id"].tolist()
    sublist = sublist[checkpoint:]
    fields = ['id', 'author', 'body',  'parent_id',
              'score', 'subreddit', 'subreddit_id', 'submission_id']
    parsing_fields = ['id', 'author', 'body',  'parent_id',
              'score', 'subreddit', 'subreddit_id']
    
    file_name = "data/" + "comment_" + sub + ".csv"
    
    if not checkpoint:
        df = pd.DataFrame(columns=fields)
        df.to_csv(file_name)
    
    for post in tqdm(sublist):
        comment_list = []
        submission = reddit.submission(post)
        submission.comments.replace_more(limit=8)
        for comment in submission.comments.list():
            to_dict = vars(comment)
            sub_dict = {field:to_dict[field] for field in parsing_fields}
            sub_dict.update({'submission' : submission.id})
            comment_list.append(sub_dict)
        df = pd.DataFrame(data = comment_list)
        df.to_csv(file_name, mode='a', header = False)
            
#     comm_data = pd.DataFrame(data = comment_list, columns=fields)
#     file_name = "data/" + sub + "_comment.csv"
#     subs_data.to_csv(file_name, index = False)

In [25]:
for sub in tqdm(subreddit_list[5:]):
    subreddit_crawler(sub, limit = 1000)

  0%|          | 0/1 [00:00<?, ?it/s]

In [100]:
for sub in subreddit_list:
    data_processing(sub)

subreddit : politics
       num_comments          score  upvote_ratio
count   1000.000000    1000.000000    1000.00000
mean    3351.287000   47589.902000       0.87574
std     2525.661836   15315.985498       0.05474
min      511.000000   28821.000000       0.65000
25%     1926.000000   36101.750000       0.85000
50%     2805.000000   43838.500000       0.89000
75%     4165.000000   55091.000000       0.91000
max    39789.000000  119394.000000       0.97000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1000 non-null   object 
 1   author        960 non-null    object 
 2   title         1000 non-null   object 
 3   selftext      9 non-null      object 
 4   num_comments  1000 non-null   int64  
 5   score         1000 non-null   int64  
 6   upvote_ratio  1000 non-null   float64
 7   url           1000 non-null   object 
dty

In [26]:
for sub in tqdm(subreddit_list[5:]):
    comment_parser(sub, checkpoint = 0)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]